# Reddit Query Tool

## What is it?
This tool lets you query information from the [Reddit Pushshift API](https://github.com/pushshift/api). With this tool you can:

- Explore how often Reddit users have posted about a given topic
- Find sample posts about a given topic
- Get top-level metrics about Reddit posts

## How can I use it?
Simply add in your query into the below fields. All fields are optional. Please ensure that all fields, including blank fields, are wrapped in single quotes ('').

Once you have added in all of your query info hit "Cell" -> "Run all" (https://screenshot.googleplex.com/mPuJGiqfKMK.png).

## More info
For questions, comments, feature requests, more information, or a friendly chat reach out to zmpollack@.

In [3]:
# Search by specific post IDs as a comma separated list. 
# Example: ids = 'abcd, wxyz, qwerty'
ids = ''

# Keyword search term. 
# Example: q = 'android'
q = ''

# Exlude a keyword in the search.
# Example: q_not: 'robots'
q_not = ''

# Search just post titles by keyword.
# Example: title = 'battery'
title = ''

# Exclude a keyword in post titles. 
# Example: titlenot = 'iPhone'
titlenot = ''

# Number of posts to be returned. Can be between 0 - 500. Default is 25.
# Exmple: size = '10'
size = ''

# Search by post author. Only one author can be specified. 
# Example: author = 'sundar'
author = ''

# Limit the search to a specfic subreddit. Only one subreddit can be specified. No need to include "r/".
# Example: subreddit = 'google'
subreddit = ''

# Search for submissions posted after a given date. Format must be mm/dd/yyyy.
# Example: after = '09/27/2019'
after = ''

# Search for submissions posted before a given date. Format must be mm/dd/yyyy.
# Example: before = '12/31/2019'
before = ''

# Restrict the search to posts based on post score as expressed using '>' or '<'.
# There should not be a space between characters.
# Example: score = '>100'
score = ''

# Limit search to posts with a given number of comments as expressed using '>' or '<'.
# There should not be a space between characters.
# Example: num_comments = '>50'
num_comments = ''

# Sort results in a specific order. Options are "asc" and "desc".
# If sort_type is not specified results will be sorted by post creation time. 
# Example: sort = 'desc'
sort = ''

# Sort results by a specific attribute. Options are "score", "num_comments", "created_utc".
# Example: sort_type = 'score'
sort_type = ''

# Return results as either a csv for import into Sheets or as readible text.
# Input True for csv output. Input False for text output
is_csv = False

In [4]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

import requests
import json
import time
import os
import urllib
import csv

def convertTime():
    global after
    global before
    pattern = '%m/%d/%Y'
    os.environ['TZ']='UTC'
    
    if after == '':
        pass
    else:
        after = str(int(time.mktime(time.strptime(after, pattern))))
    if before == '':
        pass
    else:
        before = str(int(time.mktime(time.strptime(before, pattern))))
   
def createRequest():
    global q
    global q_not
    
    if q == '':
        pass
    else:
        q = urllib.parse.quote_plus(q)
        
    if q_not == '':
        pass
    else:
        q_not = urllib.parse.quote_plus(q_not)
    
    request = 'https://api.pushshift.io/reddit/search/submission/?'
    request += 'size=' + size + '&'
    request += "ids=" + ids + "&"
    request += "q=" + q + "&"
    request += "q:not=" + q_not + "&"
    request += "title=" + title + "&"
    request += "title:not=" + titlenot + "&"
    request += "author=" + author + "&"
    request += "subreddit=" + subreddit + "&"
    request += "after=" + after + "&"
    request += "before=" + before + "&"
    request += "score=" + score + "&"
    request += "num_comments=" + num_comments + "&"
    request += "sort=" + sort + "&"
    request += "sort_type=" + sort_type + "&"
    request += "metadata=true" 
    
    return request
    
def getResponse(request):
    r = requests.get(request)
    result = json.loads(r.content)
    return result
    
def formatResponse(args):
    
    metadata = args['metadata']
    total_results = metadata['total_results']
    results_returned = metadata['results_returned']
    
    metadata_str = """
    **Info** \n \n
    Total submissions: %s \n
    Posts returned: %s \n
    *************************** \n
    """ %(total_results, results_returned)
    
    data = args['data']

    
    i = 0 
    data_str = '**Posts** \n \n'
    
    while i < results_returned:
        author = data[i]['author']
        full_link = data[i]['full_link']
        num_comments = data[i]['num_comments']
        subreddit = data[i]['subreddit']
        title = data[i]['title']
        score = data[i]['score']
        
        data_str += """
        Title : %s \n
        Author: %s \n
        Subreddit: %s \n
        Link: %s \n
        Number of comments: %s \n
        Score: %s \n
        *************************** \n
        """ %(title, author, subreddit, full_link, num_comments, score)
        
        i += 1
    
    full_str = str(metadata_str) + str(data_str)
    
    return(full_str)

def csvResponse(args):
    
    metadata = args['metadata']
    results_returned = metadata['results_returned']
    
    data = args['data'] 
    i = 0
    data_str = """
    Copy the below, paste without formatting into a Google Sheet \n
    ****************************************************************\n
    title\t author\t subreddit\t full link\t number of comments\t score
    """
    
    while i < results_returned:
        author = str(data[i]['author']).rstrip()
        full_link = str(data[i]['full_link']).rstrip()
        num_comments = str(data[i]['num_comments']).rstrip()
        subreddit = str(data[i]['subreddit']).rstrip()
        title = str(data[i]['title']).rstrip()
        score = str(data[i]['score']).rstrip()
        
        data_str += "%s\t%s\t%s\t%s\t%s\t%s\n" %(title, author, subreddit, full_link, num_comments, score)
        
        i += 1
    
    return(data_str)
    
convertTime()

request = createRequest()

response = getResponse(request)

if is_csv == True:
    print(csvResponse(response))
else:
    print(formatResponse(response))


    Copy the below, paste without formatting into a Google Sheet 

    ****************************************************************

    title	 author	 subreddit	 full link	 number of comments	 score
    get iphone x gestures on android without root	masteroduo	Android	https://www.reddit.com/r/Android/comments/ew3fes/get_iphone_x_gestures_on_android_without_root/	1	1
Samsung Galaxy A50: gestures vs. navigation "buttons" when it comes it battery life	DerLeuchtkeks	Android	https://www.reddit.com/r/Android/comments/esc4sh/samsung_galaxy_a50_gestures_vs_navigation_buttons/	0	1
Navigation gestures	lmanop	Android	https://www.reddit.com/r/Android/comments/ep1ofa/navigation_gestures/	0	1
Unpopular Opinion: I like Android Pie's gestures more than Android Q's	bjackson171	Android	https://www.reddit.com/r/Android/comments/eo57kp/unpopular_opinion_i_like_android_pies_gestures/	69	1
What do y'all think of Android 10's gestures?	AtomRidge	Android	https://www.reddit.com/r/Android/comments/en7bm8/w